In [277]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Dynamic Programming (Cont'd)


### 0-1 Knapsack

Suppose there are $n$ objects, each with a *value* $v_i$ and *weight* $w_i$. You have a "knapsack" of capacity $W$ and want to fill it with a set of objects $X \subseteq [n]$ so that $w(X) \leq W$ and $v(X)$ is maximized. 

<img src="Knapsack.jpg" width="50%">


Let $OPT(S, W)$ be an optimal solution to the Knapsack problem for a set of objects $S$ and capacity $W$.

We started with $n$ objects and capacity $W$ so we are interested in finding $OPT([n], W)$. 


If object $n$ is in the optimal solution, then <br>
<br>
$~~~~~~~~OPT([n], W)=\{n\} \cup OPT([n-1], W-w(n)).$ 
<br>
<br>
If it isn't, then <br>
<br>
$~~~~~~~~OPT([n], W) = OPT([n-1], W)$.



<br>
<br>

**Optimal Substructure for Knapsack**: For any set of objects $[n]$ and $W>0$, we have

$$v(OPT([n], W)) = \max\Big \{v(n) + v\big(OPT([n-1], W - w(n))\big), ~v(OPT([n-1], W)\big)\Big\}.$$

Let's consider the recursion tree:

<img src="knapsack_recursion_tree.jpg" width="70%">

> $\Omega(2^n)$ work and $O(n)$ span.  

### <span style="color:red">Memoization</span> - All about sharing

We compute $v(OPT(i, w))$ once and save the result for later use (e.g., in a suitable data structure). When performing memoization, we can either proceed **top-down** or **bottom-up**:



Then, we no longer have a binary tree but rather a **directed acyclic graph** or **DAG**.


<img src="knapsack_recursion_dag.jpg" width="70%">



<br>

> The number of nodes in this DAG will allow us to determine the work of this algorithm
> The longest path in the DAG will allow us to determine the span. 


### Example:

|index |value|weight|
|------|------|-----|
|0     | 10   |5    |
|1     | 6    | 3 |
|2     | 6    | 2 |


Optimal solution is 12 (second and third items) for capacity $W =5$.
<br>


<br><br>**Define a table**: number of items to include (rows) by weight (cols)
<span style="color:red">**Note**</span>: The size of table $(n+1)\times(W+1)$ when we include boundaries.

| |0 |1 |2 |3 |4 |5 |
|-|-|-|-|-|-|-|
|0|0 |0 |0 |0 |0 |0 |
|1|0 |0 |0 |0 |0 |10|
|2|0 |0 |0 |6 |6 |10|
|3|0 |0 |6 |6 |6 |**12** |





In [1]:
import random


### objects = [(10,5), (6,3), (6,2)]

## Implementation 1
def recursive_knapsack(objects, i, W):
    v, w = objects[i]
    if (i == 0):
        if (w <= W):
            return(v)
        else:
            return(0)
    else:
        if (w <= W):
            take = v + recursive_knapsack(objects, i-1, W-w)
            dont_take = recursive_knapsack(objects, i-1, W)
            return(max(take, dont_take))
        elif (W == 0):
            return(0)
        else:
            # w>W
            return(recursive_knapsack(objects, i-1, W))


In [2]:
## Implementation 2
def tabular_knapsack(objects, W):
    n = len(objects)
    # we'll rely on indices to also represent weights, so we'll index from 1...W 
    # in the weight dimension of the table
    OPT = [[0]*(W+1)]

    
    # use the optimal substructure property to compute increasingly larger solutions
    for i in range(0,n):
        OPT.append([0]*(W+1))
        v_i, w_i = objects[i]
        for w in range(W+1):
            if (w_i <= w):
                OPT[i+1][w] = max(v_i + OPT[i][w-w_i], OPT[i][w])
            else:
                OPT[i+1][w] = OPT[i-1][w] 

    return(OPT[n][W])


In [13]:
## Evaluation Stage
## Case 1
W = 5
objects = [(10,5), (6,3), (6,2)]

Ts = time.time()

print('Implementation 1:', recursive_knapsack(objects, len(objects)-1, W))
T1 = time.time()
print('The time cost for recursive version', round(T1-Ts, 5))


print('Implementation 2:', tabular_knapsack(objects, W))
T2 = time.time()
print('The time cost for recursive version', round(T2-T1, 5))


Implementation 1: 12
The time cost for recursive version 0.00041
Implementation 2: 12
The time cost for recursive version 0.0002


In [15]:

## Case 2
W = 5
objects = [(10, 5), (9.999, 3)]

Ts = time.time()

print('Implementation 1:', recursive_knapsack(objects, len(objects)-1, W))
T1 = time.time()
print('The time cost for recursive version', round(T1-Ts, 5))


print('Implementation 2:', tabular_knapsack(objects, W))
T2 = time.time()
print('The time cost for recursive version', round(T2-T1, 5))

Implementation 1: 10
The time cost for recursive version 0.00063
Implementation 2: 10
The time cost for recursive version 0.00059


In [ ]:
## Case 3
import time
W = 100
n = 500
objects = [(i, i) for i in range(1, n)]

Ts = time.time()

print('Implementation 1:', recursive_knapsack(objects, len(objects)-1, W))
T1 = time.time()
print('The time cost for recursive version', round(T1-Ts, 5))


print('Implementation 2:', tabular_knapsack(objects, W))
T2 = time.time()
print('The time cost for recursive version', round(T2-T1, 5))


### Elements of Dynamic Programming

This is what we call **dynamic programming**. The elements of a dynamic programming algorithm are:

- Optimal Substructure
- Recursion DAG

The correctness of the dynamic programming approach follows from the optimal substructure property (i.e., induction). If we can prove that the optimal substructure property holds, and that we compute a solution by correctly implementing this property then our solution is optimal.

As with divide and conquer algorithms, achieving a good work/span can be tricky. We can minimize redundant computation by memoizing solutions to all subproblems. This can be done *top-down* by saving the result of a recursive call the first time we encounter it. Or, we can compute the optimal substructure property *bottom-up* by starting with the base case(s) and working our way up.

Can we derive the number of nodes in the DAG using the optimal substructure property?


### Work and Span in Dynamic Programming

Since we memoize the solution to every distinct subproblem, the number of nodes in the DAG is equal to the number of distinct subproblems considered. 

The longest path in the DAG represents the span of our dynamic programming algorithm.


For example, 0-1 Knapsack Problem (n, W)
- There are at most $O(nW)$ nodes in this DAG, and the longest path is $O(n)$. Each node requires $O(1)$ work/span, so the work is $O(nW)$ and the span is $O(n)$. 


### Why "Dynamic Programming"?

The mathematician Richard Bellman coined the term ["dynamic programming"](https://en.wikipedia.org/wiki/Dynamic_programming) to describe the recursive approach we just showed. The optimal substructure property is sometimes referred to as a "Bellman equation." But why did he call it dynamic programming?

There is some [folklore](https://en.wikipedia.org/wiki/Dynamic_programming#History) around the exact reason. But it could possibly be because "dynamic" is a really dramatic way to describe the search weaving through the DAG. The term "programming" was used in the field of optimization in the 1950s to describe an optimization approach (e.g., linear programming, quadratic programming, mathematical programming). 


#### One More Example 


The capacity is 11, and there are 5 items with different values and weights.<br>
<img src="0-1Quiz.png" width="24%">

$$
\begin{array}{l}
\mathit{Fib}~x =   \\
~~~~\texttt{if}{}~~x \le 1~~\texttt{then}{}\\
~~~~~~~~x\\   
~~~~\texttt{else}\\
~~~~~~~~\texttt{let}{}~~(ra, rb) = (\mathit{Fib}~(x-1))~~,~~(\mathit{Fib}~(x-2))~~\texttt{in}{}\\  
~~~~~~~~~~~~ra + rb\\  
~~~~~~~~\texttt{end}{}.\\
\end{array}
$$ 